<a href="https://colab.research.google.com/github/hospino/Cursos-Platzi/blob/main/cargarROADMAPCompletoExportaDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Celda Cero: Inicialización de GEE
import ee
PROYECTO_ID = "gen-lang-client-0528470346"

try:
    # Intenta inicializar directamente con el proyecto.
    ee.Initialize(project=PROYECTO_ID)
    print("✅ Earth Engine inicializado.")

except Exception as e:
    # Si falla, es porque el token expiró. Ejecuta la autenticación de nuevo.
    print("❌ Error de inicialización. Ejecutando ee.Authenticate()...")
    ee.Authenticate()
    # Si la autenticación es exitosa, intenta inicializar de nuevo.
    ee.Initialize(project=PROYECTO_ID)
    print("✅ Autenticación y reinicialización exitosa.")

✅ Earth Engine inicializado.


In [ ]:

# Celda 2: Obtención y Procesamiento de la Imagen Sentinel-2 (Recorte por Estado)

import ee
# Asegúrate de que GEE esté inicializado (Celda Cero) antes de ejecutar esta.

# 1. Definir la ubicación y el Asset ID
# Usando el ID del Asset que subiste a tu proyecto
ASSET_ID_MERIDA = 'projects/gen-lang-client-0528470346/assets/MeridaBorde'
lon = -71.146
lat = 8.587
punto_merida = ee.Geometry.Point(lon, lat)

try:
    # Carga la geometría usando el ID del Asset.
    # Nota: El Asset ID de proyecto requiere el prefijo 'projects/'
    area_de_estudio = ee.FeatureCollection(ASSET_ID_MERIDA).geometry()
    print(f"✅ Geometría del Estado Mérida ({ASSET_ID_MERIDA}) cargada. Usando recorte político.")
except Exception as e:
    print(f"❌ ERROR: El Asset ID no es válido. Usando el círculo de 30km como respaldo. Error: {e}")
    area_de_estudio = punto_merida.buffer(30000)

# 2. Definir el rango de tiempo y Colección Sentinel-2
fecha_inicio = '2024-01-01'
fecha_fin = '2024-03-01'

# Filtros
coleccion_s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(fecha_inicio, fecha_fin) \
    .filterBounds(area_de_estudio) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

# 3. Crear el mosaico compuesto y RECORTARLO con el límite del Estado
imagen_compuesta = coleccion_s2.median().clip(area_de_estudio)

print("✅ Datos Sentinel-2 procesados (listos para el recorte poligonal).")

✅ Geometría del Estado Mérida (projects/gen-lang-client-0528470346/assets/MeridaBorde) cargada. Usando recorte político.
✅ Datos Sentinel-2 procesados (listos para el recorte poligonal).


In [ ]:
 # Celda 3: Visualización con Base Satelital (HYBRID) y Centrado Forzado

from IPython.display import display, Markdown
import geemap

# Parámetros de la ubicación de Mérida
lon = -71.146
lat = 8.587
zoom_level = 9 # Usamos un zoom ligeramente más alejado para ver todo el límite del estado

# 1. Parámetros de Visualización (Color Natural)
vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'] # Bandas Sentinel-2: Red, Green, Blue
}

# 2. Crear el objeto Mapa (Usando el basemap estable 'HYBRID')
m = geemap.Map(basemap='HYBRID')

# 3. FORZAR el centrado del mapa en Mérida (Anula el centrado predeterminado de África)
m.setCenter(lon, lat, zoom_level)

# 4. Añadir la capa de imagen satelital de GEE (Recortada al polígono)
m.addLayer(
    ee_object=imagen_compuesta,
    vis_params=vis_params,
    name='Sentinel-2 Compuesta (Recorte Poligonal)'
)

# 5. Mostrar el mapa
m.add_layer_control()
display(Markdown("### ✅ Resultado: Recorte Poligonal del Estado Mérida"))
m

### ✅ Resultado: Recorte Poligonal del Estado Mérida

Map(center=[8.587, -71.146], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

In [ ]:

# Celda 4: Cálculo y Visualización del NDVI

from IPython.display import display, Markdown

# 1. CÁLCULO DEL NDVI
# Fórmula: (NIR - Red) / (NIR + Red) -> (B8 - B4) / (B8 + B4)
# La variable 'imagen_compuesta' fue creada en la Celda 2 y ya está recortada al Estado Mérida.
ndvi = imagen_compuesta.normalizedDifference(['B8', 'B4']).rename('NDVI')

# 2. PARÁMETROS DE VISUALIZACIÓN DEL NDVI
# Usamos una paleta que va de rojo (poca o nula vegetación) a verde oscuro (vegetación densa).
ndvi_vis_params = {
    'min': -0.2, # Agua, nubes, áreas urbanas
    'max': 0.8,  # Vegetación densa y saludable
    'palette': [
        'blue',     # -0.2 (Agua)
        'white',    # 0.0 (Nieve/Nubes)
        'yellow',   # 0.1 - 0.2 (Suelo desnudo/Urbano)
        'green',    # 0.3 - 0.5 (Pastos/Bosques poco densos)
        'darkgreen' # 0.6 - 0.8 (Bosques densos y sanos)
    ]
}

# 3. AÑADIR CAPA NDVI AL MAPA EXISTENTE (m)
try:
    # La capa NDVI se superpondrá a la capa 'Sentinel-2 Compuesta'
    m.addLayer(
        ee_object=ndvi,
        vis_params=ndvi_vis_params,
        name='Índice de Vegetación (NDVI)'
    )

    # 4. Mostrar el mapa con la nueva capa
    m.add_layer_control()
    display(Markdown("### ✅ Resultado: Mapa con Capa NDVI Añadida"))
    m # Muestra el mapa interactivo con la nueva capa NDVI

except NameError:
    display(Markdown("❌ **Error:** Asegúrate de ejecutar la Celda 3 (Visualización) primero para crear el objeto del mapa (`m`)."))

### ✅ Resultado: Mapa con Capa NDVI Añadida

In [ ]:

# Celda 5: Cálculo del Valor Promedio del NDVI

from IPython.display import display, Markdown

# Asegúrate de usar la variable 'ndvi' calculada en la Celda 4.

# 1. Definir los parámetros para el cálculo zonal
# 'reducer': ee.Reducer.mean() calcula el promedio de todos los píxeles.
# 'geometry': area_de_estudio es el polígono del Estado Mérida.
# 'scale': 10 metros es la resolución nativa de Sentinel-2 para las bandas B4 y B8.
# 'maxPixels': 1e9 para evitar errores de límites si la región es muy grande.
stats = ndvi.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=area_de_estudio,
    scale=10,
    maxPixels=1e9
)

# 2. Obtener el valor del diccionario de resultados
# El resultado de GEE es un 'Server-side object'. Usamos .get('NDVI').getInfo() para traer el valor numérico.
ndvi_promedio = stats.get('NDVI').getInfo()

# 3. Presentar el resultado
display(Markdown("### 📊 Resultado de la Métrica NDVI"))
print(f"El valor promedio del NDVI para el Estado Mérida (Ene-Feb 2024) es:")
print(f"NDVI Promedio: {ndvi_promedio:.4f}")

# 4. Interpretación Rápida
if ndvi_promedio > 0.5:
    display(Markdown("✅ **Interpretación:** Este valor sugiere una cobertura vegetal densa y saludable en la mayor parte del Estado (típico de zonas montañosas y boscosas)."))
elif ndvi_promedio > 0.3:
    display(Markdown("⚠️ **Interpretación:** Este valor sugiere vegetación moderada, quizás una mezcla de bosques, pastizales y áreas agrícolas."))
else:
    display(Markdown("❌ **Interpretación:** Este valor bajo podría indicar una fuerte presencia de nubes residuales, áreas urbanas grandes, o zonas de suelo desnudo o nieve (dependiendo de la altitud)."))

### 📊 Resultado de la Métrica NDVI

El valor promedio del NDVI para el Estado Mérida (Ene-Feb 2024) es:
NDVI Promedio: 0.6187


✅ **Interpretación:** Este valor sugiere una cobertura vegetal densa y saludable en la mayor parte del Estado (típico de zonas montañosas y boscosas).

In [ ]:

# Celda 6: Cálculo de Superficie de Nieve/Hielo (NDSI)

# 1. CÁLCULO DEL NDSI
# Usamos las bandas B3 (Green) y B11 (SWIR1)
ndsi = imagen_compuesta.normalizedDifference(['B3', 'B11']).rename('NDSI')

# 2. DEFINIR LA NIEVE (SEGMENTACIÓN)
# El umbral estándar para nieve/hielo es NDSI > 0.4.
nieve = ndsi.gt(0.4) # Genera una imagen binaria (1 si es nieve, 0 si no)

# 3. CÁLCULO DEL ÁREA EN KILÓMETROS CUADRADOS
# 'ee.PixelArea()' genera una imagen donde cada píxel tiene su área en m².
area_nieve = nieve.multiply(ee.Image.pixelArea()) # Mantiene solo el área donde hay nieve (valor=1)

# 'reduceRegion' suma el área total de todos los píxeles de nieve
stats_area = area_nieve.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=area_de_estudio,
    scale=10,
    maxPixels=1e9
)

# 4. Obtener y convertir el resultado a km²
area_m2 = stats_area.get('NDSI').getInfo()
area_km2 = area_m2 / 1e6 # Convertir de metros cuadrados a kilómetros cuadrados

# 5. Visualización de la capa de nieve (opcional, ayuda a verificar)
nieve_vis = {'palette': ['cyan']}
m.addLayer(nieve.updateMask(nieve), nieve_vis, 'Nieve/Hielo (NDSI > 0.4)')

# 6. Presentar el resultado
display(Markdown("### 🏔️ Resultado de Cobertura de Nieve"))
print(f"La superficie total de Nieve/Hielo (NDSI > 0.4) visible en el Estado Mérida (Ene-Feb 2024) es:")
print(f"Área de Nieve: {area_km2:.2f} km²")

# 7. Mostrar el mapa con la nueva capa (si no lo hiciste en la Celda 4)
m.add_layer_control()
m

### 🏔️ Resultado de Cobertura de Nieve

La superficie total de Nieve/Hielo (NDSI > 0.4) visible en el Estado Mérida (Ene-Feb 2024) es:
Área de Nieve: 38.42 km²


Map(bottom=15836.02084350586, center=[9.275396177356708, -71.3619231618941], controls=(WidgetControl(options=[…

In [ ]:

# Celda 7: Cálculo y Área de Cuerpos de Agua (NDWI)

from IPython.display import display, Markdown

# 1. CÁLCULO DEL NDWI
# Usamos las bandas B3 (Green) y B8 (NIR)
ndwi = imagen_compuesta.normalizedDifference(['B3', 'B8']).rename('NDWI')

# 2. DEFINIR EL AGUA (SEGMENTACIÓN)
# El umbral estándar para agua es NDWI > 0.0, ya que el agua absorbe NIR y refleja Green.
agua = ndwi.gt(0.0) # Genera una imagen binaria (1 si es agua, 0 si no)

# 3. CÁLCULO DEL ÁREA EN KILÓMETROS CUADRADOS
# Usamos ee.PixelArea() y la geometría del Estado Mérida.
area_agua = agua.multiply(ee.Image.pixelArea())
stats_area_agua = area_agua.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=area_de_estudio,
    scale=10,
    maxPixels=1e9
)

# 4. Obtener y convertir el resultado a km²
area_m2_agua = stats_area_agua.get('NDWI').getInfo()
area_km2_agua = area_m2_agua / 1e6 # Convertir de metros cuadrados a kilómetros cuadrados

# 5. Visualización de la capa de agua (opcional)
agua_vis = {'palette': ['#0000FF']} # Azul vibrante
m.addLayer(agua.updateMask(agua), agua_vis, 'Cuerpos de Agua (NDWI > 0.0)')

# 6. Presentar el resultado
display(Markdown("### 💧 Resultado de Cobertura de Agua"))
print(f"La superficie total de Cuerpos de Agua (NDWI > 0.0) en el Estado Mérida (Ene-Feb 2024) es:")
print(f"Área de Agua: {area_km2_agua:.2f} km²")

# 7. Mostrar el mapa con la nueva capa
m.add_layer_control()
m

### 💧 Resultado de Cobertura de Agua

La superficie total de Cuerpos de Agua (NDWI > 0.0) en el Estado Mérida (Ene-Feb 2024) es:
Área de Agua: 130.88 km²


Map(bottom=7988954.0623168945, center=[8.550168437349301, -70.99801182725061], controls=(WidgetControl(options…

In [ ]:

# Celda 8: Exportación a Google Drive

from IPython.display import display, Markdown
from google.colab import drive
import ee

# 1. MONTAR GOOGLE DRIVE
display(Markdown("### 🔗 Paso 1: Autenticación de Google Drive"))
print("Por favor, sigue el enlace para autenticar Google Drive y permitir que Colab guarde archivos.")
drive.mount('/content/drive')

### 🔗 Paso 1: Autenticación de Google Drive

Por favor, sigue el enlace para autenticar Google Drive y permitir que Colab guarde archivos.
Mounted at /content/drive


In [ ]:

# 2. CREAR LA TAREA DE EXPORTACIÓN EN EARTH ENGINE

# Definimos un nombre descriptivo para el archivo GeoTIFF
archivo_nombre = 'Merida_Sentinel2_EneFeb2024_Analisis'

# Usamos la imagen compuesta recortada y el polígono del Estado para definir la exportación.
tarea = ee.batch.Export.image.toDrive(
    image=imagen_compuesta, # La imagen ya recortada y filtrada
    description=archivo_nombre,
    folder='GEE_Exports_Merida', # Se creará esta carpeta en tu Drive
    fileNamePrefix=archivo_nombre,
    region=area_de_estudio.bounds().getInfo()['coordinates'], # Usamos el límite del polígono
    scale=10, # Resolución nativa de 10 metros de Sentinel-2
    crs='EPSG:4326',
    maxPixels=1e13
)

# Iniciar la tarea de exportación
tarea.start()

display(Markdown("### ✅ Tarea de Exportación Creada"))
print(f"El archivo '{archivo_nombre}.tif' se está exportando a Google Drive.")
print("La tarea se ejecutará en segundo plano. Sigue su progreso en el Code Editor de GEE.")

### ✅ Tarea de Exportación Creada

El archivo 'Merida_Sentinel2_EneFeb2024_Analisis.tif' se está exportando a Google Drive.
La tarea se ejecutará en segundo plano. Sigue su progreso en el Code Editor de GEE.
